In [1]:
!pip install -q tf-nightly-2.0-preview

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [3]:
import tensorflow as tf
import datetime

In [4]:
# Clear any logs from previous runs
!rm -rf ./logs/ 
!mkdir ./logs/

In [5]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

11493376/11490434 [==============================] - 1s 0us/step


In [6]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

W0614 08:13:59.682744 140616197744448 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/math_grad.py:1251: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 8s 131us/sample - loss: 0.2191 - accuracy: 0.9357 - val_loss: 0.1017 - val_accuracy: 0.9695
Epoch 2/5
60000/60000 [==============================] - 7s 122us/sample - loss: 0.0970 - accuracy: 0.9699 - val_loss: 0.0782 - val_accuracy: 0.9747
Epoch 3/5
60000/60000 [==============================] - 7s 124us/sample - loss: 0.0697 - accuracy: 0.9778 - val_loss: 0.0730 - val_accuracy: 0.9776
Epoch 4/5
60000/60000 [==============================] - 7s 122us/sample - loss: 0.0529 - accuracy: 0.9829 - val_loss: 0.0693 - val_accuracy: 0.9782
Epoch 5/5
60000/60000 [==============================] - 8s 133us/sample - loss: 0.0437 - accuracy: 0.9852 - val_loss: 0.0695 - val_accuracy: 0.9767


In [8]:
%tensorboard --logdir ./logs/fit

In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

In [10]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [11]:
# Define our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [12]:
def train_step(model, optimizer, x_train, y_train):
  with tf.GradientTape() as tape:
    predictions = model(x_train, training=True)
    loss = loss_object(y_train, predictions)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss)
  train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
  predictions = model(x_test)
  loss = loss_object(y_test, predictions)

  test_loss(loss)
  test_accuracy(y_test, predictions)

In [13]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = './logs/gradient_tape/' + current_time + '/train'
test_log_dir = './logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [14]:
model = create_model() # reset our model

EPOCHS = 5

for epoch in range(EPOCHS):
  for (x_train, y_train) in train_dataset:
    train_step(model, optimizer, x_train, y_train)
  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_dataset:
    test_step(model, x_test, y_test)
  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))

  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.24521180987358093, Accuracy: 92.97000122070312, Test Loss: 0.11512196063995361, Test Accuracy: 96.59000396728516
Epoch 2, Loss: 0.10480816662311554, Accuracy: 96.83999633789062, Test Loss: 0.08753574639558792, Test Accuracy: 97.3699951171875
Epoch 3, Loss: 0.07205667346715927, Accuracy: 97.8316650390625, Test Loss: 0.07498450577259064, Test Accuracy: 97.63999938964844
Epoch 4, Loss: 0.05357808619737625, Accuracy: 98.375, Test Loss: 0.061746615916490555, Test Accuracy: 98.1199951171875
Epoch 5, Loss: 0.042375460267066956, Accuracy: 98.67832946777344, Test Loss: 0.06627801060676575, Test Accuracy: 97.89999389648438


In [18]:
!kill 220
%tensorboard --logdir ./logs/gradient_tape/